In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import pandas as pd

# Load dataset
df = pd.read_csv(r"C:\Users\dirga\Python_AI\Machine_Learning\latihan_lks\Iris.csv")
df.drop(columns=["Id"], inplace=True)

# Encode labels
class_mapping = {label: idx for idx, label in enumerate(df['Species'].unique())}
df['Species'] = df['Species'].map(class_mapping)

X = df.drop('Species', axis=1).values
y = df['Species'].values

# Implementasi Decision Tree dari main.ipynb
class TreeNode():
    def __init__(self, data, feature_idx, feature_val, prediction_probs, information_gain) -> None:
        self.data = data
        self.feature_idx = feature_idx
        self.feature_val = feature_val
        self.prediction_probs = prediction_probs
        self.information_gain = information_gain
        self.feature_importance = self.data.shape[0] * self.information_gain
        self.left = None
        self.right = None

class DecisionTree():
    def __init__(self, max_depth=4, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf

    def _entropy(self, class_probabilities):
        return sum([-p * np.log2(p) for p in class_probabilities if p > 0])

    def _class_probabilities(self, labels):
        unique, counts = np.unique(labels, return_counts=True)
        return counts / counts.sum()

    def _data_entropy(self, labels):
        return self._entropy(self._class_probabilities(labels))

    def _split(self, data, feature_idx, feature_val):
        mask_below_threshold = data[:, feature_idx] < feature_val
        return data[mask_below_threshold], data[~mask_below_threshold]

    def _find_best_split(self, data):
        min_part_entropy = float('inf')
        best_split = None

        for idx in range(data.shape[1] - 1):
            feature_vals = np.unique(data[:, idx])
            for val in feature_vals:
                g1, g2 = self._split(data, idx, val)
                part_entropy = self._data_entropy(g1[:, -1]) + self._data_entropy(g2[:, -1])
                if part_entropy < min_part_entropy:
                    min_part_entropy = part_entropy
                    best_split = (g1, g2, idx, val)
        return best_split

    def _create_tree(self, data, depth):
        if depth > self.max_depth or data.shape[0] <= self.min_samples_leaf:
            labels, counts = np.unique(data[:, -1], return_counts=True)
            pred_probs = np.zeros(len(np.unique(y)))
            pred_probs[labels.astype(int)] = counts / counts.sum()
            return TreeNode(data, None, None, pred_probs, 0)
        
        best_split = self._find_best_split(data)
        if not best_split:
            return None
        
        g1, g2, feature_idx, feature_val = best_split
        node = TreeNode(data, feature_idx, feature_val, None, 0)
        node.left = self._create_tree(g1, depth + 1)
        node.right = self._create_tree(g2, depth + 1)
        return node

    def train(self, X_train, y_train):
        train_data = np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)
        self.tree = self._create_tree(train_data, 0)

    def predict(self, X_set):
        predictions = []
        for sample in X_set:
            node = self.tree
            while node and node.feature_idx is not None:
                if sample[node.feature_idx] < node.feature_val:
                    node = node.left
                else:
                    node = node.right
            predictions.append(np.argmax(node.prediction_probs))
        return predictions

# Train Decision Tree
model = DecisionTree(max_depth=4, min_samples_leaf=1)
model.train(X, y)

# GUI Functions
def classify_iris():
    try:
        sl = float(entry_sepal_length.get())
        sw = float(entry_sepal_width.get())
        pl = float(entry_petal_length.get())
        pw = float(entry_petal_width.get())

        input_data = np.array([[sl, sw, pl, pw]])
        prediction = model.predict(input_data)[0]
        species_name = list(class_mapping.keys())[list(class_mapping.values()).index(prediction)]
        result_label.config(text=f"Hasil Prediksi: {species_name}", fg="green", font=("Helvetica", 14, "bold"))
    except ValueError:
        messagebox.showerror("Input Error", "Silahkan masukkan angka yang valid dengan titik sebagai pemisah desimal.")

def reset_fields():
    entry_sepal_length.delete(0, tk.END)
    entry_sepal_width.delete(0, tk.END)
    entry_petal_length.delete(0, tk.END)
    entry_petal_width.delete(0, tk.END)
    result_label.config(text="")

# GUI setup
root = tk.Tk()
root.title("Klasifikasi Data Iris dengan Decision Tree")
root.geometry("820x580") 
root.resizable(False, False)  
root.configure(bg="#FFFFFF")

header = tk.Label(root, text="Lomba Kompetensi Siswa - Artificial Intelligence 2025", font=("Helvetica", 20, "bold"), bg="#FFFFFF")
header.pack(pady=16)

title = tk.Label(root, text="Aplikasi Prediksi Species Bunga Iris", font=("Helvetica", 18, "bold"), bg="#FFFFFF")
title.pack(pady=(5, 8))

sub_header = tk.Label(root, text="Klasifikasi Data Iris dengan Decision Tree", font=("Helvetica", 14), bg="#FFFFFF")
sub_header.pack(pady=5)

frame = tk.Frame(root, bg="#FFFFFF")
frame.pack(pady=10)

labels = ["Sepal Length (cm):", "Sepal Width (cm):", "Petal Length (cm):", "Petal Width (cm):"]
entries = []

for i, text in enumerate(labels):
    tk.Label(frame, text=text, font=("Helvetica", 12), bg="#FFFFFF").grid(row=i, column=0, padx=5, pady=5, sticky="w")
    entry = tk.Entry(frame, font=("Helvetica", 12), width=21)
    entry.grid(row=i, column=1, padx=5, pady=5)
    entries.append(entry)

entry_sepal_length, entry_sepal_width, entry_petal_length, entry_petal_width = entries

tk.Label(root, text="Silahkan memasukkan angka, gunakan titik (.) sebagai pemisah desimal\n", font=("Helvetica", 11), bg="#FFFFFF", fg="red").pack()

button_frame = tk.Frame(root, bg="#FFFFFF")
button_frame.pack(pady=10)

reset_btn = tk.Button(button_frame, text="Reset", command=reset_fields, bg="#f44336", fg="white", font=("Helvetica", 12, "bold"), width=14, height=1)
reset_btn.grid(row=0, column=0, padx=5)

predict_btn = tk.Button(button_frame, text="Prediksi", command=classify_iris, bg="#4CAF50", fg="white", font=("Helvetica", 12, "bold"), width=14, height=1)
predict_btn.grid(row=0, column=1, padx=8)

result_label = tk.Label(root, text="", font=("Helvetica", 12, "bold"), bg="#FFFFFF")
result_label.pack(pady=10)

# Footer
footer_text = """
Tim AI SMK Negeri 4 Malang 2025
Farrel Dirga Purnama & Ayu Nilam Sari
"""

footer_label = tk.Label(root, text=footer_text, font=("Helvetica", 11), bg="#FFFFFF", anchor="w", justify="left")
footer_label.place(x=10, rely=1.0, anchor="sw")

root.mainloop()
